In [49]:
!pip install -q peft transformers datasets accelerate torch evaluate fastapi uvicorn huggingface_hub
!pip install

In [55]:
# !pip install -q peft transformers datasets accelerate torch evaluate fastapi uvicorn huggingface_hub
!pip install evaluate

  Using cached evaluate-0.4.3-py3-none-any.whl.metadata (9.2 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 5.2 MB/s eta 0:00:00


In [56]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from datasets import load_dataset
from peft import PromptTuningConfig, get_peft_model, TaskType
import evaluate

In [82]:
dataset = load_dataset("amazon_polarity", split="train[:1000]")  # Using only 2000 samples

In [83]:
model_name = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Preprocess the dataset
def preprocess(example):
    encoding = tokenizer(example["content"],return_tensors="pt", truncation=True, padding="max_length", max_length=128)
    return {
        "input_ids": encoding["input_ids"],  # Ensure input_ids are provided
        "attention_mask": encoding["attention_mask"],
        "labels": example["label"],  # Ensure labels are present
    }

dataset = dataset.map(preprocess, batched=True)
dataset = dataset.train_test_split(test_size=0.2)

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [84]:
peft_config = PromptTuningConfig(
    task_type=TaskType.SEQ_CLS,
    num_virtual_tokens=5,
    prompt_tuning_init="random",
    num_layers=3,
    token_dim=768,
    num_attention_heads=6
)

# Load base model
base_model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)

# Apply prompt tuning
model = get_peft_model(base_model, peft_config)
model.print_trainable_parameters()

#Freeze base model weights to only train prompt
for param in model.base_model.parameters():
    param.requires_grad = False

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


trainable params: 595,970 || all params: 67,550,980 || trainable%: 0.8823


In [85]:
#Set training arguments (Small batch size for Colab)
training_args = TrainingArguments(
    per_device_train_batch_size=8,  # ⬇️ Reduce batch size to avoid memory issues
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    output_dir="./results",
    save_strategy="no",
    logging_dir="./logs",
    logging_steps=10,
    do_train=True,
    do_eval=True,
    evaluation_strategy="epoch",
    report_to="none",
)

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [86]:
#Train model
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
)

trainer.train()

Epoch,Training Loss,Validation Loss
1,0.696800,0.694305
2,0.693600,0.694303
3,0.701400,0.694305


TrainOutput(global_step=300, training_loss=0.6947906812032064, metrics={'train_runtime': 21.4785, 'train_samples_per_second': 111.739, 'train_steps_per_second': 13.967, 'total_flos': 80571853209600.0, 'train_loss': 0.6947906812032064, 'epoch': 3.0})

In [106]:
#save model
model_path = "sentiment_prompt_tuned_model"
model.save_pretrained(model_path)
tokenizer.save_pretrained(model_path)

('sentiment_prompt_tuned_model/tokenizer_config.json',
 'sentiment_prompt_tuned_model/special_tokens_map.json',
 'sentiment_prompt_tuned_model/vocab.txt',
 'sentiment_prompt_tuned_model/added_tokens.json',
 'sentiment_prompt_tuned_model/tokenizer.json')

In [107]:
#check if model folder in colab
import os

model_path = "/content/sentiment_prompt_tuned_model"
print("Exists:", os.path.exists(model_path))
print("Contents:", os.listdir(model_path) if os.path.exists(model_path) else "Folder not found")

Exists: True
Contents: ['README.md', 'tokenizer.json', 'vocab.txt', 'special_tokens_map.json', 'tokenizer_config.json', 'adapter_model.safetensors', 'adapter_config.json']


In [112]:
import shutil
# Zip the model folder
shutil.make_archive(model_path, 'zip', model_path)

# Download the zipped model
from google.colab import files
files.download(model_path + ".zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [89]:
#with argmax
def predict_sentiment(review):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")  # Detect GPU
    model.to(device)  # Ensure model is on the same device

    inputs = tokenizer(review, return_tensors="pt", truncation=True, padding="max_length", max_length=128)
    inputs = {k: v.to(device) for k, v in inputs.items()}  # ✅ Move inputs to same device

    with torch.no_grad():
        outputs = model(**inputs)

    prediction = torch.argmax(outputs.logits, dim=-1).item()
    return "Positive" if prediction == 0 else "Negative"

# ✅ Test inference
print(predict_sentiment("This product is amazing!"))
print(predict_sentiment("Worst purchase ever."))


Positive
Negative


In [98]:
#with softmax
import torch.nn.functional as F

def predict_sentiment(review):
    inputs = tokenizer(review, return_tensors="pt", truncation=True, padding=True, max_length=128).to(device)
    outputs = model(**inputs)
    probs = F.softmax(outputs.logits, dim=-1)  # Convert logits to probabilities
    confidence, prediction = torch.max(probs, dim=-1)  # Get class with highest probability

    return ("Positive" if prediction.item() == 1 else "Negative"), confidence.item()

print(predict_sentiment("It is an amazing product, I used for 3months and reviewing it!"))


('Positive', 0.5106593370437622)


In [ ]:
#Testing uploaded model
from transformers import AutoModelForSequenceClassification, AutoTokenizer

model = AutoModelForSequenceClassification.from_pretrained("vidya55/sentiment_prompt_tuned_model")
tokenizer = AutoTokenizer.from_pretrained("vidya55/sentiment_prompt_tuned_model")

In [99]:
from sklearn.metrics import precision_recall_fscore_support

In [ ]:
# Zero-shot model (Pretrained without tuning)
zero_shot_model_name = "distilbert-base-uncased-finetuned-sst-2-english"
zero_shot_model = AutoModelForSequenceClassification.from_pretrained(zero_shot_model_name)
zero_shot_tokenizer = AutoTokenizer.from_pretrained(zero_shot_model_name)

# Fine-tuned model
fine_tuned_model_name = "your_huggingface_model"
fine_tuned_model = AutoModelForSequenceClassification.from_pretrained(fine_tuned_model_name)
fine_tuned_tokenizer = AutoTokenizer.from_pretrained(fine_tuned_model_name)


In [ ]:
#define predict function
def predict_sentiment(model, tokenizer, text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)
    with torch.no_grad():
        outputs = model(**inputs)
    logits = outputs.logits
    probs = torch.softmax(logits, dim=1)
    pred_class = torch.argmax(probs, dim=1).item()

    sentiment = "Positive" if pred_class == 1 else "Negative"
    confidence = probs[0, pred_class].item()

    return sentiment, confidence


In [ ]:
#define test dataset
test_reviews = [
    "This product is amazing!",
    "Worst purchase ever.",
    "I am in love with food!",
    "It's a wonderful product!",
    "I hate this product!",
    "It is an amazing product, I used it for 3 months and reviewing it!"
]


In [ ]:
#Compare Zero-Shot vs. Fine-Tuned Predictions
print("Zero-Shot Model Predictions:")
for review in test_reviews:
    sentiment, confidence = predict_sentiment(zero_shot_model, zero_shot_tokenizer, review)
    print(f"Review: {review} -> {sentiment} ({confidence:.4f})")

print("\nFine-Tuned Model Predictions:")
for review in test_reviews:
    sentiment, confidence = predict_sentiment(fine_tuned_model, fine_tuned_tokenizer, review)
    print(f"Review: {review} -> {sentiment} ({confidence:.4f})")


In [ ]:
#Compute Precision, Recall, and F1-score
from sklearn.metrics import classification_report

# Actual labels (1=Positive, 0=Negative)
actual_labels = [1, 0, 1, 1, 0, 1]

# Predictions from both models
zero_shot_preds = [1 if predict_sentiment(zero_shot_model, zero_shot_tokenizer, r)[0] == "Positive" else 0 for r in test_reviews]
fine_tuned_preds = [1 if predict_sentiment(fine_tuned_model, fine_tuned_tokenizer, r)[0] == "Positive" else 0 for r in test_reviews]

# Compute Precision, Recall, F1-score
print("Zero-Shot Model Evaluation:")
print(classification_report(actual_labels, zero_shot_preds))

print("\nFine-Tuned Model Evaluation:")
print(classification_report(actual_labels, fine_tuned_preds))
